In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm_notebook as tqdm
import regex as re
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
## Load Data

path = 'Data/Cleaned/xrp_cleaned_date.pkl'
randata_df = pd.read_pickle(path)

path = 'Data/Raw/goodUserCrypto.pkl'
gooddata_df = pd.read_pickle(path)

In [3]:
toremove = dict.fromkeys((ord(c) for c in u'\xa0\n\t'))


WEB_URL_REGEX = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""

# Call with dataframe column
def regularize_urls(df_col):
    return df_col.str.replace(WEB_URL_REGEX, '_url_')

In [4]:
gooddata_df['text'] = gooddata_df['text'].str.translate(toremove)

## Regularize urls
randata_df['text'] = randata_df.apply(lambda row : row['text'] + ' ' + row['usernameTweet'], axis=1)
gooddata_df['text'] = gooddata_df.apply(lambda row : row['text'] + ' ' + row['usernameTweet'], axis=1)

# URLS
randata_df['text'] = regularize_urls(randata_df['text'])
gooddata_df['text'] = regularize_urls(gooddata_df['text'])

#Mentions
randata_df['text'] = randata_df['text'].apply(lambda t : re.sub(r'@[A-Za-z0-9]+','@',t))
gooddata_df['text'] = gooddata_df['text'].apply(lambda t : re.sub(r'@[A-Za-z0-9]+','@',t))

# Hashtags
# data_df['text'] = data_df['text'].apply(lambda t : re.sub("#[^a-zA-Z]", "#", t))
randata_df['text'] = randata_df['text'].apply(lambda t : t.replace("#", " "))
gooddata_df['text'] = gooddata_df['text'].apply(lambda t : t.replace("#", " "))

#lowercase
randata_df['text'] = randata_df['text'].str.lower()
gooddata_df['text'] = gooddata_df['text'].str.lower()

In [5]:
## Create Spam List

spamWords = ['ico', 'freetoken', 'token', 'airdrop', 'airdrops', 'rippleprice_', 'bigpumpgroup', 'bounty', 'usd', 'korea price', 'binance', 'current price', \
            'cryptopricexrp', 'cryptobot', 'coinstats', 'coinpricenow', 'cryptogulp', 'ripplebot1h', 'bigdata', 'RippleMarket','CryptoGulp', 'ripplebot_cs', \
            'cryptopricebot','VirtualMoneyBot','coinstats','JustFactsNL', 'free', 'trx', 'XrpTicker', 'aWebAnalysis', 'scotlarock727', '1bitcoinkaclira', \
            'pureinvestments']
spamWordsCon = ("|").join(spamWords)

# spamUsers = ['848989770923601922','945716745192574977','944577421008875521','883664986270314498','946477078907518976','901444148690960384','919958774500249600', '726437948527398912','598343001']
# spamUsersCon = ("|").join(spamWords)

## Label Spam
randata_df['Spam'] = randata_df['text'].str.contains(spamWordsCon)
gooddata_df['Spam'] = False

In [6]:
gooddata_df.head()

,ID,datetime,is_reply,is_retweet,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,has_media,medias,Spam
0,8.794407e+17,2017-06-26 16:46:13,1.0,0.0,0.0,1.0,0.0,"lost me on that one, nick think it's the term...",/PJSanderson/status/879440684624871424,168649848,PJSanderson,NaN,NaN,False
1,9.830839e+17,2018-04-08 16:47:12,1.0,0.0,3.0,0.0,0.0,"not only that, but freedom of speech is only p...",/SeamusWalsh2/status/983083861763473408,343861762,SeamusWalsh2,NaN,NaN,False
2,9.895482e+17,2018-04-26 12:54:16,0.0,0.0,4.0,0.0,1.0,a fantastic video from the archives - coinsc...,/coinscrum/status/989548224171167744,1196956998,coinscrum,1.0,[https://t.co/ra4K4ln6yx],False
3,9.626423e+17,2018-02-11 05:59:37,0.0,0.0,48.0,8.0,7.0,every morning i read through the posts from ...,/whatbitcoindid/status/962642274596335616,893818632089763840,whatbitcoindid,NaN,NaN,False
4,9.915655e+17,2018-05-02 02:30:13,0.0,0.0,2.0,0.0,0.0,"""ask them why they idle therewhile we suffer, ...",/poetrypotion/status/991565507051577344,62152913,poetrypotion,NaN,NaN,False


In [7]:
# Visualize bad tweet IDS

spammers = {}
for i in range(len(randata_df['Spam'])):
    if randata_df['Spam'][i] == True:
        if randata_df['usernameTweet'][i] in spammers:
            spammers[randata_df['usernameTweet'][i]] += 1
        else:
            spammers[randata_df['usernameTweet'][i]] = 1

In [8]:
# spammers['1004118978']

In [9]:
spamIDs = []

for key in spammers:
    if spammers[key] > 50:
        spamIDs.append(key)

In [10]:
spamIDs

['RippleMarket',
 'Matthix191',
 'CryptoGulp',
 'ripplebot_cs',
 'cryptopricebot',
 'frierav',
 'ripplebot1h',
 'VirtualMoneyBot',
 'rippleprice_',
 'coinstats',
 'CryptoPriceXRP',
 'JustFactsNL',
 'coinpricenow',
 'aWebAnalysis',
 'WHardingKY',
 'Crypto_Monkeyo',
 'scotlarock727',
 'XrpTicker',
 '1bitcoinkaclira',
 'SCoinAnalytics',
 'pureinvestments']

In [11]:
randata_df['text'][randata_df['usernameTweet'] == 'pureinvestments']

606      $9k:  bitcoin  price sees first consecutive we...
1861     if you missed our weekly  podcast , don't desp...
2081     don't miss this weeks podcast! sunday at 12 pm...
2271       amazon  granted patent for streaming data ma...
2723       avast  exposes the risks of  cryptomining  t...
3319     don't miss this weeks podcast! sunday at 12 pm...
3357     crazy amount of volume traded in one hour!\n\n...
3657     if you missed our weekly  podcast , don't desp...
4573     if you missed our weekly  podcast , don't desp...
4948     top 100  cryptocurrencies  in 4 words or less\...
6197       nasa  researches  ethereum   blockchain  tec...
6401       robinhood ’s first tentative steps into  cry...
7247     if you missed our weekly  podcast , don't desp...
7248       bitmain  gets go-ahead for us bitcoin  minin...
7427       mastercard  eyes  blockchain  for fighting f...
7479     don't fall for this  phishing  attempt!  coinb...
7630       huawei  unveils hyperledger-powered  blockch.

In [12]:
## Make training list

training_df = gooddata_df
training_df = training_df.append(randata_df[randata_df['Spam'] == True])

In [13]:
training_df['Spam'].sum()

13402

In [14]:
## preprocess data

def preProcess(X, Y, method = 'BBoW'):
    
    print('Pre-processing files...')
    
    # create lemmatizer
    wnl = WordNetLemmatizer()
    
    # create stopWords
    stopWords = set(stopwords.words('english'))
    
    # initialize output variables
    vocab = {'1'}
    vocab.remove('1')
    data_x = []
    data_y = []
    N = 0 #number of documents
    doc_count = {}
    
    # for each file..
    for i in range(len(X)):
        
        label = Y[i]
        data = X[i]
            
        hold = re.compile(r'\b\w{1,}\b', re.IGNORECASE)
        words = hold.findall(data)
        # tokenize words with punctuation that would not be found with the above regex search
        hold2 = re.compile(r'\b\w{1,}[.!?]', re.IGNORECASE)
        words2 = hold2.findall(data)
        words.extend(words2)
                
        ## treat data
        doc_dict = {}
        N = N + 1
        for word in words:
            
            # lemmatize
            word = wnl.lemmatize(word)
                
            if word not in stopWords:
                vocab.add(word)

                if method == 'BBoW':
                    doc_dict[word] = 1

                if (method == 'CCoW') or (method == 'TF-IDF'):
                    if word in doc_dict:
                        doc_dict[word] = doc_dict[word] + 1
                    else:
                        doc_dict[word] = 1
                        if method == 'TF-IDF':
                            if word in doc_count:
                                doc_count[word] = doc_count[word] + 1
                            else:
                                doc_count[word] = 1
            
        #if doc_dict:
        data_x.append(doc_dict)
        data_y.append(label)
    
    # generate idfs for each wordss
    if method == 'TF-IDF':
        for i in range(len(data_x)):
            for key in data_x[i]:
                idf = np.log10(N / doc_count[key])
                data_x[i][key] = data_x[i][key] * idf
            
    print('Done')
    return data_x, data_y, vocab

In [15]:
def genPofY(labels):
    # count the number of labels present to generate PofY distribution
    
    PofY = {}
    N = len(labels)
    for label in labels:
        if label in PofY:
            PofY[label] = PofY[label] + 1
        else:
            PofY[label] = 1
            
    for key in PofY:
        PofY[key] = PofY[key]/N
    
    return PofY

In [16]:
def train(label_list, data_x, data_y, k, vocab):
    # train model

    prob_dist = {}
    denom = {}
    
    # initialize prob_dist & denom for each label
    for label in label_list:
        prob_dist[label] = {}
        denom[label] = 0
     
    # create running sum of words present in each doc per label
    for doc in range(len(data_x)):
        for word in data_x[doc]:
            label = data_y[doc]
            val = data_x[doc][word]
            if word in prob_dist[label]:
                prob_dist[label][word] += val + k
            else:
                prob_dist[label][word] = val + k
            denom[label] += val
    
    # divide each word in each label by the number of words present
    for label in prob_dist:
        denom[label] += len(vocab)*k
        for word in vocab:
            if word in prob_dist[label]:
                prob_dist[label][word] = np.log10(prob_dist[label][word] / denom[label])
            else:
                # if word in vocab is not present in the label use laplace smoothing to create an arbitrarily low 
                # probability
                prob_dist[label][word] = np.log10(k / denom[label])
        
    return prob_dist

In [17]:
def test(data_x_test, data_y_test, label_list, prob_dist):
    # test predictions against actual labels and return accuracy
    
    correct = 0.0;
    cnt = 0
    for doc in range(len(data_x_test)):
        label_prob = []
        for label in label_list:
            hold = 0
            for word in data_x_test[doc]:
                if word in prob_dist[label]:
                    hold += prob_dist[label][word]
            hold += np.log10(PofY_train[label])
            label_prob.append(hold) 
        label_guess = label_list[np.argmax(label_prob)]
        if label_guess == data_y_test[doc]:
            correct += 1.0
        cnt += 1
        
    acc = correct / cnt
    return acc

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(np.asarray(training_df['text']), np.asarray(training_df['Spam']), test_size = 0.2)

In [19]:
## Verify Naive Bayes

# pre-process data with each word representation scheme, train, and test 
data_x_train, data_y_train, vocab_train = preProcess(X_train, Y_train, method = 'TF-IDF')

data_x_test, data_y_test, vocab_test = preProcess(X_test, Y_test, method = 'TF-IDF')

# generate P of Y
PofY_train = genPofY(data_y_train)

# make label list
label_list = []
for label in PofY_train:
    label_list.append(label)

# laplace smoothing constant
k = .1

# train 
prob_dist = train(label_list, data_x_train, data_y_train, k, vocab_train)

# Test Accuracy
test_acc = test(data_x_test, data_y_test, label_list, prob_dist)

# Train Accuracy
train_acc = test(data_x_train, data_y_train, label_list, prob_dist)
    
print('Test Accuracy: ')
print(test_acc)
print('Train Accuracy: ')
print(train_acc)

Pre-processing files...
Done
Pre-processing files...
Done
Test Accuracy: 
0.9848976940565118
Train Accuracy: 
0.9950470932120818


In [20]:
# add probability that a tweet is spam

def predict(model, UX):
    # predict UY for a given model and UX

    # Prediction confidences
    PY = []
    UY = []
    
    # for each doc generate label probabilites and select the label with the highest probability
    for doc in range(len(UX)):
        label_prob = []
        for label in label_list:
            hold = 0
            for word in UX[doc]:
                if word in model[label]:
                    hold += model[label][word]
            hold += np.log10(PofY_train[label])
            label_prob.append(hold) 
            
        # append Confidence and prediction sets
        PY.append(np.max(label_prob))
        UY.append(label_list[np.argmax(label_prob)])
        
    return UY, PY

In [21]:
## generate data to classify

data2class_df = randata_df[randata_df['Spam'] == False]

In [22]:
# pre-process data with each word representation scheme, train, and test 
data_x, data_y, vocab = preProcess(np.asarray(training_df['text']), np.asarray(training_df['Spam']), method = 'TF-IDF')

# generate P of Y
PofY_train = genPofY(data_y)

# make label list
label_list = []
for label in PofY_train:
    label_list.append(label)

# laplace smoothing constant
k = .1

# train 
prob_dist = train(label_list, data_x, data_y, k, vocab)

UY, PY = predict(prob_dist, np.asarray(data2class_df['text']))

Pre-processing files...
Done


In [23]:
len(data2class_df['text'])

12033

In [24]:
len(UY)

12033

In [25]:
sum(UY)

336

In [26]:
data2class_df.reset_index(inplace=True)

In [27]:
data2class_df['Y_Pred'] = UY
data2class_df['PY'] = PY

C:\Users\Sam\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Sam\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [28]:
for i in range(len(data2class_df)):
    # if (UY[i] == True) and (PY[i] > -200):
    if (UY[i] == True):
        print(data2class_df['text'][i])
        print('')
        print('')

  ripple   xrp   xrpthestandard   allvor https:// _url_ /status/986275965679620098   … xrpthestandard7


今さら聞けない「仮想通貨」に「ブロックチェーン」って？〈aera〉（aera dot.） https:// _url_ 0417-00000016-sasahi-sci   … xrp0514


2000円

 ripple   xrp   リップル _url_ t99938854


  xrp   ripple   xrpthestandard    btc https:// _url_ us/985799671368110080   … sameralmasri25


  xrp   ripple   xrpthestandard   0doubt https:// _url_ 988469707106729984   … xrpmoonboiz


a history of  xrp  &  ripple  part 1 20112013  http:// bitme.site/posts/75116   domenitsadu80b


hourly crypto price check:
1. bitcoin (btc)  |  $8809.01  |  24hr: 5.44%
2. ethereum (eth)  |  $605.271  |  24hr: 4.82%
3.  ripple  ( xrp )  |  $0.906752  |  24hr: 7.43%
4. bitcoin cash (bch)  |  $1094.15  |  24hr: 11.11%
5. eos (eos)  |  $11.0985  |  24hr: 12.6% cyptobot


going into overload!!! :)
 ripple   xrpthestandard   xrparmy   $ xrp   https:// _url_ s/987035305009131520   … frankieenzo


  xrp   ripple   xrpthestandard https:// _url_ status/986220906

今日の暴騰 1230 neoscoin (neos)  ripple  ( xrp )  http:// bitme.site/posts/74800   melyssaxzrch


  リップル   xrp   ripple   https:// _url_ us/986212183406080005   … jinpapa38


up, up, up! 
 xrpthestandard   xrp   ripple   https:// _url_ /status/985500388102033408   … nordicann


00時 24時間で増えたフォロワー数(増加率)
 bcd  2925(+7.9%) @_ 
 xvg  1266(+0.4%) @ 
 xrp  934(+0.1%) @ 
 act  933(+0.8%) @ 
 kcs  807(+0.3%) @ 
 btc  770(+0.1%) @ 
 mco  588(+0.9%) @_card 
 xlm  543(+0.2%) @ grasshoppercoin


  ripple   $ xrp   https:// _url_ us/984606958014664704   … cryptobaa


latest  ripple  ( xrp ) details: 

 price: $ 0.671105
 price (btc): ฿ 0.00008077
 24h volume: $ 610,996,000.00
 market cap: $ 26,255,503,317.00
 change 1h: -0.22%
 change 24h: +5.96%
 change 7d: +34.74%

 shared via coinmarketapp by programonks:
 android:  https:// _url_   josua_1236


really? https:// _url_ s/985177358264025088   … manuelittoc


  xrp   ripple   https:// _url_ us/984821850239848451   … oooalxooo


今日の暴騰 1230 neoscoin (neos)

5. eos (eos)  |  $11.4508  |  24hr: 19.61% cyptobot


https:// _url_ s/986214738970402816   … xrptrump


sbi  ripple  asiaが主導する証券コンソーシアムが発足！swftとの提携でリップルの価格が上昇？  https:// _url_ nsortium/   … niko_1208_205


crypto update:
 bitcoin   8275.14  +0.51%
 ether   565.40  +2.07%
 bitcoincash   965.27  +0.99%
 ripple   0.829  +11.82%
 litecoin   145.50  +0.27%
 btc   eth   bch   xrp   ltc igsquawk


xrp  @  !!  https:// _url_ status/986355820391751681   … vegasrenegade


vote for  ripple   xrpthestandard   xrp   ripple   https:// _url_ /985461386691203073   … mikemello40


  ripple  and  xrp  getting traction  https:// _url_ 988469707106729984   … alttrades


【暗号資産】rippleリップル総合552【 xrp 】  http:// _url_ cryptocoin/1523573647/   … kattyu69691919


おいらの仮想通貨 : 【 ripple 】リップル 557【 xrp 】   http:// _url_ es/8711876.html   … _url_ hidebo0011


圧倒的に xrp がセクシーですね。
 xrp   ripple   https:// _url_ tus/986072134274334721   … yamamu1717


fuddesk did it again!! https:// _url_ s/987030152872783873   … ripplep



  cryptocurrency  prices (23 apr am - 24h):

 bitcoin : $8,939.00 (+0.34%)
 ethereum : $646.90 (+3.13%)
 ripple : $0.8789 (-1.39%)
 eos : $11.70 (+1.07%)
 litecoin : $152.42 (+1.27%)

 $ crypto   $ btc   $ eth   $ xrp   $ ltc   $ ada   $ eos   $ ada   $ xlm   $ neo 

 http:// _url_   _url_ bitcoin2140com


おいらの仮想通貨 : 【暗号資産】rippleリップル総合559【 xrp 】   http:// _url_ es/8753372.html   … _url_ hidebo0011


ふふふ。
 xrp   ripple   https:// _url_ us/986762903113023488   … yamamu1717


hourly crypto price check:
1. bitcoin (btc)  |  $8512.25  |  24hr: 3.08%
2. ethereum (eth)  |  $592.628  |  24hr: 6.3%
3.  ripple  ( xrp )  |  $0.871069  |  24hr: 16.6%
4. bitcoin cash (bch)  |  $1127.36  |  24hr: 16.87%
5. eos (eos)  |  $11.1586  |  24hr: 19.67% cyptobot


おいらの仮想通貨 : 【暗号資産】rippleリップル総合553【 xrp 】   http:// _url_ es/8655220.html   … _url_ hidebo0011


  ripple   xrp 
 https:// _url_ tus/986162917954420737   … sameralmasri25


xrp が@ に追加されます。
 xrp   ripple   https:// _url_ /986150879421255680   … yam

  xrp   ripple   https:// _url_ tatus/987071528633155584   … lucifer999999


hehe  0doubt   xrpthestandard https:// _url_ atus/985828350403973120   … ripplepandaxrp


check it out >>  7731  silver   deal  1000x gold & silver  ripple  commemorative round collectors coin  xrp  coins gifts  http:// _url_ 200-19255-0/1?ff3=2&toolid=10039&campid=5338246737&item=192422004121&vectorid=229466&lgeo=1   …  via @ _url_ soka_deals


a history of  xrp  &  ripple  part 1 20112013  http:// bitme.site/posts/75116   y37edint


crypto update:
 bitcoin   8221.59  +0.4%
 ether   534.64  +2.28%
 bitcoincash   892.77  +0.68%
 ripple   0.719  +0.6%
 litecoin   140.82  +1.08%
 btc   eth   bch   xrp   ltc igsquawk


  ripple   xrp   https:// _url_ us/987394685437034496   … davearini


  xrp   ripple   xrpthestandard https:// _url_ tatus/988687239096958976   … xrpmoonboiz


今日のまとめ速報！ 「【 ripple 】リップル546【 xrp  】」元スレ1 ：承認済み名無しさん：2018/04/09(月) 18:11:51.30 id: http:// _url_   ...   http:// _url_   omatome2c


help t

5. eos (eos)  |  $10.8047  |  24hr: -3.44% cyptobot


今日の暴騰 1230 neoscoin (neos)  ripple  ( xrp )  http:// bitme.site/posts/74800   honkogairon




In [29]:
## View non-spam

for i in range(len(data2class_df['text'])):
    if (UY[i] == False):
        print(data2class_df['text'][i])
        print('')
        print('')

new post (could  ripple   xrp  really reach $10.00 in 39 days?) has been published on  -  https:// _url_   _url_ excelvisibility


did you know that you can buy, sell, store, convert and transfer @   xrp  on  http:// _url_       in the uk, eu countries, australia, kenya, nigeria and south africa. you can use fiat currency or convert other coins and exchange them for  ripple  . coindirectcom


walmart and moneygram partnership could pump  ripple  ( xrp ) @ https:// _url_ neygram-partnership-could-pump-ripple-xrp/   … vandecrypto


this is not fake news. look at our references. we didn’t mislead anyone, we clearly stated their not using  ripple   xrp  however the inter ledger can lead to a mature relationship that may lead to them using  xrp . sniperstube


xrp  &amp; ripple: the world’s financial infrastructure is using ripplenet, _url_. _url_ pple/xrp-ripple-the-worlds-financial-infrastructure-is-using-ripplenet/   … btc_current


yo, adrian!  ripple  has reached $ 0.750  https:// _url

pullback to $0.58 likely, dips lower could find bids. overall, looks set to test $0.77  _url_ coindeskmarkets


@ ripple  explained in 1 photo  ripple   xrp   xrpthestandard _url_ xrp2m


xrapid requires the use of  xrp . it's just not something the customers have to know about. much like customers of the internet don't have to worry about packets of data and where it's coming from as long as they can see what they want in their web browser. xrptrump


now @  on the hands of @  like many others,@ revolution for all  money transfers.
 xrpcommunity   xrp   ripple   crypto   criptomonedas   blockchaintechnology   criptonoticias   citibank   banamex  @   followback https:// _url_ s/986193674693361664   … waynebanaf


_url_. _url_ am/2018/04/20/bitcoin-is-the-religion-but-ripple-would-pack-higher-return/ 60886141504b   … 

 ripple   xrp   xrpthstandard xrp_news


ripple  partnership with financial messaging service may boost it’s prices -  http:// _url_ ship-with-financial-messaging-service

crypto update: bitcoin (btc), ethereum (eth), and  ripple  ( xrp ) storm higher again -  https:// _url_   viabitcoin


the bullish breakout above $0.70 was seen as hoped for in this article.  higher highs are ahead for  ripple   xrp .  https:// _url_ ealing-its-hidden-potentials-sentiment-analysis-april-18/   … e_hawkinsjr


if  ripple  dumps hard (ignoring the escrow of course), it will crush  xrp 's price, eroding confidence in  ripple 's motivations, driving investors and their clients away.   xrp  is mostly where it is today because investors believe in  ripple 's vision, not just because of  xrp  as a coin galgitron


「危機」の後にiota（miota）( ripple (リップル）仮想通貨情報局)
 https:// virtualcurrency.news/66249   

 仮想通貨   btc   xrp   eth   lisk   草コイン   cryptocurrency v_c_news_jp


ripple  was a credit network before it was a company or coin.

see, same words can have different meanings in different time periods! xrptrump


  crypto   top5 

 ripple   $ xrp  +19%  

time to 

 bitcoin   ethereum

【リップル】xrpは将来1000円が限界ぐらいだと思う【 ripple 】  http:// _url_   kakugentomeigen


ripple  price forecast: has the much-awaited  xrp  rally started? -  https:// _url_    via @ mariuskarma


  ripple  price prediction end of 2018?

 xrp   ripple   bitcoin   cryptos   alts   altcoins   altcoinseason   blockchain   blockchaintechnology   cryptonews   altcoinexchange   btc   eth   bch lastcryptojedi


i liked a @  video  http:// _url_     ripple   xrp : entering the golden era the takeover is upon us ermelindapenaf3


nice to see the market is waking up to the bulls coming back as the bears go elsewhere to find honey.  xrpthestandard   xrp   ripple   $ xrp 

 https:// _url_ ning-streak-continues-under-best-choices-for-long-term/   … claytonlloyd


thank you @  please dont stress over it im sure you have enough on your plate. the work yourself and  ripple  have been doing should allow me to buy as many shirts i could ever need ;) bankxrp


once you’ve abandoned security you can “upgrade” a blockchain

 $ btc   eth   $ eth   ripple   btc   bitcoin   ethereum   $ xrp   bitcoincash   altcoins cryptoamb


ripple  price forecast: has the much-awaited  xrp  rally started? -  http:// _url_ 3a%2f%_url_%2fcryptocurrency%2fripple%2fxrp-price-forecast-has-the-much-awaited-rally-started%2f%3futm_campaign%3dshareaholic%26utm_medium%3dtwitter%26utm_source%3dsocialnetwork&r=1   …  via @ jesuis_alfredo


ripple  is solving the biggest business problem on the planet.  this is a logical result if their efforts are successful. hunter_bick


ask yourself why so many use the word "revolution" when describing or discussing  xrp  and  ripple . i dont think it is a coincidence.  https:// _url_ /986711614295552000   … fourthjohnxrp


  xrp   ripple   リップル 
end 'wild west' days of cryptocurrencies,  ripple  urges uk regulators' | via @ _url_. _url_ 04/15/end-wild-west-days-cryptocurrencies-ripple-urges-uk-regulators/   … srgft


how far  xrp  ( ripple ) can go in this year?  _url_ ɔɔ.cc/qqxqhk    via  http:/

...

cryptocurrencies need a distribution method.  mining has proven a centralizing wasteful mechanism.  pos rewards the rich.   ripple 's approach is strategic targeted dissemination.   ripple 's  xrp  holdings are %-wise much less than the centralization of bitcoin's top unknown holders. galgitron


why  ripple 's  xrp  outperformed the other top cryptos this week...  _url_ btcethereum


he likes  xrp  lol its kool i like abra/ltc he dont like abra i dont like  xrp  either way the banks are going to find away to survive  ripple   xrp  is there life blood. i own some just in case it blow up.  i love ltc dont care for bch but i got some in abra paypay90000


http:// _url_    new  ripple  coin 24k gold plated physical  xrp  coin collectible with acrylic case  http:// _url_    - http:// _url_   _url_ smartmoneym


to the moon !!!
 crypto   bitnodes   londoncoin1000   tothemoon   ripple   xrp   bitcoin   btc   ethereum   eth   bitcoincash   bch   litecoin   ltc   dreambig   forex   money   sto

why  ripple 's  xrp  outperformed the other top cryptos this week  http:// _url_   _url_ wolfctrader


new venezuelan exchange cryxto launches for  ripple  ( xrp ) in bolivars  https:// _url_   cryptoflashn1


o my word!  this is where @  will be the leader.  billions upon billions being transferred cross-border at massive costs.  the future is so bright we are going to need high grade sunglasses!   0doubt   xrp   https:// _url_ status/988402925096587265   … rpienaar5


retweeted  xrp _news (@ _news ):

 xrp  is now priced at $0.904, up nearly 100% in the past 2 weeks alone.

 xrp   xrpthestandard   ripple joneblaze0420


i have honestly tried woth that telegram group, for a few months i was selecting serious people from the group and trying to get them to come to twitter or  xrp  chat. that place is ground 0 for  xrp  fud ekpethedragon


shameless shill! clintdebeer


time for correction  xrp   btc   ripple 

@  @  @ _xrp1  @  @  

@  r&a talks
 https:// _url_   

@  r&a channel
 http

why  ripple 's  xrp  outperformed the other top cryptos this week via @ http:// _url_   jmrozada


_url_ rpienaar5


仮想通貨リップル( ripple / xrp )の特徴・送金の仕組み・将来性。詳細を徹底解説!!  https:// cryptoccy.tokyo/ripple-descrip tion   … 
 xrp   ripple enrich_life


amazon partnership speculation high for  ripple  ( xrp ) as markets go crazy @ https:// _url_ ship-speculation-high-for-ripple-xrp-as-markets-go-crazy/   … hjvsch


  ripple  has partenered with 75 banks according to  citi 

 xrp   0doubt   xrparmy _url_ rossmacfirdeen


@ bgarlinghouse  you tube alex cobb, amazing talented young bloke advising the world daily - get him onboard some way as his young, just needs direction to become a professional  ripple  online speaker. i love helping the younger generations xrp001


ripples  financial statement has nothing to do with  xrp , they use their software money for  ripple  labs.  xrp  money goes back into the  xrp  ecosystem, not towards  ripple  labs, we know this because we can see where  xrp  goes 

3 reasons why starbucks will accept  ripple  ( xrp ) in 2018  oracle times  http:// bitme.site/posts/74090   sanvalsercte


  xrp 
 ripple  partnership with international financial messaging service could boost price  https:// _url_ -partnership-international-financial-messaging-service-boost-price/   … coinlady33


ripple  to place 55 billion  xrp  in escrow to ensure certainty of total  xrp  supply  http:// _url_   _url_ bitcryptoprice


xrp  price is en route to break the $1 resistance

 https:// _url_ n-route-to-break-the-1-resistance/   … 

 cryptocurrency   altcoins   ripple   xrp   xrparmy   xrptothemoon   cryptotrading   cryptoinvesting   cryptomarket   cryptoasset _url_ cryptoargals


why? hmmm personally i feel gold looks nicer, no? ripplepandaxrp


ripple ’s ceo and founder of arrington  xrp  to talk cryptocurrency at disrupt sf

to read more latest news visit  https:// _url_ /   

 bradgarlinghouse   xrp   michaelarrington   ripple   cryptocurrencies   cryptocurrency   bloc

  please share  tosha_yan


ripple     xrp  channel  live price chart  chat  interviews  brad garlinghouse   support 247  いまさら聞けない仮想通貨  http:// bitme.site/posts/74174   delmamflrd


ripple ’s cost efficiency solutions for banks: a breakdown -  https:// ripplenews.tech/2018/04/19/rip ples-cost-efficiency-solutions-for-banks-a-breakdown/   …   xrp   xrpthestandard   ripple xrp_chrisinger


one factor that really drew me to  $ xrp  is the fact that  ripple  employees are constantly working hard to expand access to the ledger. as we speak, @  is researching diverse approaches to gateways. https:// _url_ s/985544375664078848   … rabbitkickclub


why  ripple ’s  xrp  outperformed the other top cryptos this week  http:// _url_   _url_ cryptograbber


is this the week for  ripple  ( xrp )?  https:// _url_ ek-for-ripple-xrp/   … woggieeee


ripple 's interleger protocol and apple pay?
 xrp   applepay   ripple   xrp   xrpthestandard https:// _url_ pple-xrp-interledger-as-banking-becomes-easier-w

In [30]:
data2class_df.head()

,index,ID,datetime,is_reply,is_retweet,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,has_media,medias,rounded_dateTime,Spam,Y_Pred,PY
0,0,9.885302e+17,2018-04-23 17:29:08,0.0,0.0,0.0,0.0,0.0,new post (could ripple xrp really reach $1...,/ExcelVisibility/status/988530235493748736,815322320,ExcelVisibility,NaN,NaN,2018-04-23 18:00:00,False,False,-224.231813
1,1,9.847927e+17,2018-04-13 09:57:41,0.0,0.0,0.0,0.0,0.0,"did you know that you can buy, sell, store, co...",/coindirectcom/status/984792745666580481,915453520924545025,coindirectcom,1.0,[https://t.co/h1CTJHGwPs],2018-04-13 10:00:00,False,False,-397.801096
2,5,9.852602e+17,2018-04-14 16:55:20,0.0,0.0,0.0,0.0,0.0,walmart and moneygram partnership could pump ...,/vandecrypto/status/985260236238934018,943489871758405632,vandecrypto,1.0,[https://t.co/LxwR2chVOy],2018-04-14 17:00:00,False,False,-240.390867
3,6,9.860333e+17,2018-04-16 20:07:13,1.0,0.0,0.0,1.0,0.0,this is not fake news. look at our references....,/sniperstube/status/986033300136316928,921966736303362048,sniperstube,NaN,NaN,2018-04-16 21:00:00,False,False,-358.042842
4,9,9.851089e+17,2018-04-14 06:54:01,0.0,0.0,1.0,0.0,0.0,xrp &amp; ripple: the world’s financial infra...,/btc_current/status/985108911312093184,952620879443320832,btc_current,1.0,[https://t.co/dKr58pbYhj],2018-04-14 07:00:00,False,False,-350.440039


In [31]:
labeled_df = pd.DataFrame()
labeled_df['text'] = data2class_df['text'][data2class_df['Y_Pred']==False]

In [32]:
## Find most common words

def top_kwords(data_df, vec, k):
    X = vec.fit_transform(data_df['text'].values)
    labels = vec.get_feature_names()
    
    return pd.DataFrame(columns = labels, data = X.toarray()).sum().sort_values(ascending = False)[:k]

In [33]:
vec = CountVectorizer(decode_error = 'ignore', stop_words = 'english')
X = vec.fit_transform(labeled_df['text'].values)

In [34]:
top_words = top_kwords(labeled_df, vec, 40)

top_words

xrp                 14319
ripple              12473
_url_               11118
https                4195
bitcoin              2364
http                 2184
crypto               1942
btc                  1807
eth                  1586
cryptocurrency       1426
price                1388
ethereum             1365
xrpthestandard       1235
news                 1112
blockchain           1097
week                  695
litecoin              688
new                   634
2018                  631
market                614
xrparmy               576
just                  564
says                  548
ltc                   525
santander             486
cryptos               477
outperformed          423
like                  417
security              409
big                   396
bch                   382
eos                   368
partnership           368
value                 367
bitcoincash           366
xlm                   363
securities            355
analysis              347
cryptocurren

In [35]:
labeled_df['text'][labeled_df['text'].str.contains('xlm')]

42       last week correlation among big cryptocurrenci...
64       3 partnerships that prove stellar (xlm) will o...
65       ripple   xrp  will soon be the king of the cry...
94       what is your 3 best coins? write comment :)\n\...
180      $ str   $ xlm   str   xlm  just broke 4,030 sa...
233      iota ladestation vorgestellt  https:// _url_ o...
290      ripple ( xrp ), stellar(xlm), & dentacoin(dcn)...
319        btc   cryptocurrency   ethereum   eth   lite...
334      bitcoin (btc) could reverse exuberant  ripple ...
360        cryptocurrency  prices (24 apr - 24h):\n\n b...
402      start bitcoin nuvoo mining in this link:  http...
409      haha, pump and dump? you guys are crazy. unfol...
421      ok, this is it! \n\n xlm   $ xlm   str   $ str...
474      when your folio is negative but you still hodl...
477      bitcoin (btc) could reverse exuberant  ripple ...
495      why  stellar  (xlm) is overtaking  ripple  ( x...
544      ripple ( xrp ), stellar(xlm), & dentacoin(dcn).